In [1]:
import numpy as np 
import pandas as pd 

In [2]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('../input/janatahack-independence-day-2020-ml-hackathon/train.csv')
test = pd.read_csv('../input/janatahack-independence-day-2020-ml-hackathon/test.csv')

In [4]:
test['ALL'] = test['TITLE'] + test['ABSTRACT']
train['ALL'] = train['TITLE'] + train['ABSTRACT']

In [5]:
cols_ignore = ['ID','TITLE','ABSTRACT']
cols_y = ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']
cols_X = list(set(train.columns) - set(cols_ignore) - set(cols_y))

In [6]:
topic_labels = train[cols_y]

In [7]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [8]:
X = []
sentences = list(train["ABSTRACT"])
for sen in sentences:
    X.append(preprocess_text(sen))
#sentences2 = list(train["TITLE"])
#X2 = []
#for sen in sentences2:
#    X2.append(preprocess_text(sen))
#print(X1,X2)
#X = train[['TITLE','ABSTRACT']]
y = topic_labels.values

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)

In [12]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('../input/glove6b200d/glove.6B.200d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 200))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

FileNotFoundError: [Errno 2] No such file or directory: '../input/glove-global-vectors-for-word-representation/glove.6B.200d.txt'

In [ ]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 200, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(6, activation='sigmoid')(LSTM_Layer_1)
#LSTM_Layer_2 = Dense(50)(dense_layer_1)
#dense_layer_2 = Dense(6, activation='relu')(LSTM_Layer_2)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['acc'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_split=0.2)

In [ ]:
score = model.evaluate(X_val, y_val, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
X_test = test['ALL']
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
y_test = model.predict(X_test)

In [ ]:
y_test

In [ ]:
y_test_2 = np.where(y_test>0.3,1,0)
y_test_2

In [ ]:
sub = pd.DataFrame(y_test_2)
sub['ID'] = test['ID']
sub = sub.set_index('ID')
sub.columns = cols_y
sub = sub.to_csv('sub_av.csv')
sub